In [23]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

### Load Results

In [26]:
# Results is a dictionary access elements with results[expt_num][data_type][features_used] 
# expt_num -> [1,2,3,4] | data_types -> ['Wake','N1', 'N2', 'N3', 'REM'] | features_used --> ['regional' , 'regions_combined' , 'all_feats']
expt_nums = [1,2,3,4] 
data_types = ['Wake','N1', 'N2', 'N3', 'REM']
features_used = ['regional' , 'regions_combined' , 'all_feats']

#Max Frequency Features 1.Static , 2.Minimal TS Features , 3.Efficient TS Features
results = joblib.load('static_frequency_feats_results_2.pkl')
results = joblib.load('TS_Fresh_minimal_frequency_feats_results.pkl')
results = joblib.load('All_results_max_frequency_feats_efficient_TS_Fresh.pkl')

#BPW Features 1.Static (Not added), 2.Minimal TS Features , 3.Efficient TS Features
bpw_path = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/'

results = joblib.load(bpw_path + 'All_results_bpw_feats_minimal_TS_Fresh.pkl')
results = joblib.load(bpw_path + 'All_results_bpw_feats_efficient_TS_Fresh.pkl')


##Combined BPW and Max Frequecy Features 1.Static, 2.Minimal TS Features, 3.Efficient TS Features
core_path_combined = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Combine_Max_Freq_Power_Band/'

results = joblib.load(core_path_combined + 'static_combined_BPW_frequency_feats_results.pkl')
results = joblib.load(core_path_combined + 'All_results_bpw_plus_max_freq_minimal_TS_Fresh.pkl')
results = joblib.load(core_path_combined + 'All_results_bpw_plus_max_freq_efficient_TS_Fresh.pkl')


### Expt 1

In [27]:
expt_num = 1 

features =  'regions_combined'
features = 'all_feats'
features = 'regional'
for features in ['regional' , 'regions_combined' , 'all_feats']:
    print('\n' + features + '\n')
    for data_type in data_types:
        print('\n' + data_type + '\n')
        results_df = results[expt_num][data_type][features]
        
        res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
        
        sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
        sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
        sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))
        
        df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
        df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
        df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
        
        display(df)


regional


Wake



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.733333,0.733333,0.450000,0.500000,0.533333,0.416667,0.516667,0.466667,0.483333,0.500000,0.500000,0.583333,0.516667,0.600000
0,DT,0.616667,0.550000,0.466667,0.533333,0.266667,0.516667,0.500000,0.433333,0.500000,0.450000,0.616667,0.533333,0.616667,0.583333
0,Ada_B,0.750000,0.750000,0.550000,0.633333,0.533333,0.500000,0.550000,0.483333,0.550000,0.516667,0.566667,0.600000,0.616667,0.566667



N1



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.8125,0.7625,0.7625,0.7875,0.7625,0.7875,0.8125,0.7625,0.8125,0.8125,0.7625,0.7500,0.7875,0.775
0,DT,0.8000,0.7000,0.6875,0.7875,0.6625,0.6125,0.6375,0.6625,0.5625,0.7625,0.6875,0.6625,0.8000,0.750
0,Ada_B,0.7500,0.6625,0.7500,0.7500,0.6500,0.7125,0.7500,0.7000,0.5875,0.7375,0.6750,0.7125,0.6375,0.625



N2



...

,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.559722,0.559722,0.559722
0,DT,0.451389,0.451389,0.402778
0,Ada_B,0.558333,0.558333,0.558333



N3



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.569444,0.569444,0.569444
0,DT,0.538889,0.538889,0.525000
0,Ada_B,0.450000,0.450000,0.450000



REM



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.6500,0.650,0.6500
0,DT,0.4625,0.450,0.4625
0,Ada_B,0.5250,0.525,0.5250


### Expt 2

In [28]:
expt_num = 2

features =  'regions_combined'
features = 'all_feats'
features = 'regional'
for features in ['regional' , 'regions_combined' , 'all_feats']:
    print('\n' + features + '\n')
    for data_type in data_types:
        print('\n' + data_type + '\n')
        results_df = results[expt_num][data_type][features]
        
        res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
        
        sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
        sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
        sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))
        
        df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
        df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
        df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
        
        display(df)


regional


Wake



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.737500,0.737500,0.630357,0.589286,0.580357,0.412500,0.407143,0.407143,0.326786,0.473214,0.448214,0.523214,0.507143,0.491071
0,DT,0.601786,0.553571,0.435714,0.367857,0.480357,0.475000,0.330357,0.435714,0.601786,0.567857,0.539286,0.446429,0.500000,0.462500
0,Ada_B,0.712500,0.712500,0.330357,0.487500,0.423214,0.317857,0.448214,0.437500,0.500000,0.541071,0.487500,0.548214,0.603571,0.514286



N1



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.755556,0.607778,0.650000,0.618889,0.671111,0.703333,0.670000,0.670000,0.702222,0.651111,0.736667,0.684444,0.755556,0.694444
0,DT,0.757778,0.561111,0.631111,0.757778,0.616667,0.524444,0.750000,0.655556,0.647778,0.658889,0.573333,0.652222,0.636667,0.555556
0,Ada_B,0.861111,0.541111,0.575556,0.662222,0.487778,0.626667,0.861111,0.593333,0.514444,0.607778,0.556667,0.555556,0.572222,0.513333



N2



...

,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.520000,0.520000,0.520000
0,DT,0.510000,0.471111,0.510000
0,Ada_B,0.531111,0.531111,0.531111



N3



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.384444,0.384444,0.384444
0,DT,0.491111,0.491111,0.458889
0,Ada_B,0.574444,0.574444,0.574444



REM



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.404444,0.404444,0.404444
0,DT,0.521111,0.521111,0.435556
0,Ada_B,0.465556,0.465556,0.465556


### Expt 3

In [29]:
expt_num = 3

features =  'regions_combined'
features = 'all_feats'
features = 'regional'
for features in ['regional' , 'regions_combined' , 'all_feats']:
    print('\n' + features + '\n')
    for data_type in data_types:
        print('\n' + data_type + '\n')
        results_df = results[expt_num][data_type][features]
        
        res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
        
        sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
        sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
        sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))
        
        df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
        df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
        df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
        
        display(df)


regional


Wake



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.500758,0.500758,0.312121,0.346970,0.357576,0.196212,0.240152,0.223485,0.152273,0.286364,0.232576,0.340909,0.340152,0.179545
0,DT,0.390152,0.312879,0.304545,0.331818,0.295455,0.275758,0.312879,0.259848,0.384848,0.390152,0.302273,0.375758,0.337121,0.339394
0,Ada_B,0.393182,0.364394,0.215152,0.335606,0.284091,0.277273,0.296970,0.296212,0.322727,0.321212,0.333333,0.345455,0.393182,0.278030



N1



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.507143,0.470000,0.463333,0.457143,0.478095,0.450000,0.499524,0.442381,0.458095,0.507143,0.422381,0.401905,0.408095,0.410000
0,DT,0.565714,0.337619,0.388095,0.388095,0.359048,0.492381,0.565714,0.444286,0.339524,0.429048,0.303333,0.408095,0.322857,0.421905
0,Ada_B,0.473333,0.393333,0.374286,0.429048,0.386190,0.443333,0.473333,0.400952,0.438095,0.452857,0.310952,0.380952,0.344762,0.401429



N2



...

,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.360476,0.360476,0.360476
0,DT,0.443333,0.407619,0.443333
0,Ada_B,0.382381,0.382381,0.382381



N3



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.241905,0.241905,0.241905
0,DT,0.326667,0.326667,0.276667
0,Ada_B,0.208571,0.208571,0.208571



REM



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.340659,0.340659,0.340659
0,DT,0.347802,0.340659,0.347802
0,Ada_B,0.362637,0.362637,0.362637


### Expt 4

In [30]:
expt_num = 4

features =  'regions_combined'
features = 'all_feats'
features = 'regional'
for features in ['regional' , 'regions_combined' , 'all_feats']:
    print('\n' + features + '\n')
    for data_type in data_types:
        print('\n' + data_type + '\n')
        results_df = results[expt_num][data_type][features]
        
        res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
        
        sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
        sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
        sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))
        
        df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
        df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
        df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
        
        display(df)


regional


Wake



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.384848,0.384848,0.294697,0.354545,0.338636,0.292424,0.264394,0.275758,0.231061,0.347727,0.293182,0.248485,0.376515,0.339394
0,DT,0.338636,0.337879,0.249242,0.330303,0.318939,0.257576,0.267424,0.233333,0.338636,0.284091,0.312879,0.233333,0.258333,0.278030
0,Ada_B,0.470455,0.354545,0.283333,0.362879,0.328788,0.281818,0.258333,0.293939,0.470455,0.321970,0.286364,0.302273,0.303030,0.301515



N1



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.471905,0.429048,0.393810,0.408095,0.409048,0.450952,0.471905,0.458095,0.458571,0.443810,0.416190,0.414762,0.358095,0.401905
0,DT,0.394762,0.281905,0.301905,0.311429,0.293810,0.393333,0.394762,0.353810,0.283810,0.343810,0.190476,0.339048,0.318571,0.330952
0,Ada_B,0.395238,0.254762,0.289524,0.302857,0.288095,0.345238,0.395238,0.351905,0.332381,0.366667,0.260952,0.373333,0.379048,0.388095



N2



...

,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.317619,0.317619,0.317619
0,DT,0.311429,0.281905,0.311429
0,Ada_B,0.380000,0.380000,0.380000



N3



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.192857,0.192857,0.192857
0,DT,0.270000,0.270000,0.249524
0,Ada_B,0.278095,0.278095,0.278095



REM



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.435165,0.435165,0.435165
0,DT,0.304945,0.296703,0.304945
0,Ada_B,0.339011,0.339011,0.339011
